In [8]:
import subprocess
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
from src.MOEDA import MOEDA
from src import variation

N = 5

# L = [5, 10, 20, 15, 30, 60] # number of (discrete) variables
L = [15, 30, 60] # number of (discrete) variables
elitsmSettings = [True]
# populationSizes = [
# 	[50, 50, 50, 50, 50, 50],
# 	[200, 500, 500, 500, 500, 500]
# ]
populationSizes = [
	[50, 50, 50]
]

for i, l in enumerate(L):
	hypervolume_results_per_setting = []
	for j, elitsm in enumerate(elitsmSettings):
		hypervolume_results = []
		for i in range(N):
			populationSize = populationSizes[j][i]
			EA = MOEDA(populationSize = populationSize,
					numberOfVariables = l,
					numberOfEvaluations = 10**4,
					fitnessFunction = 'knapsack',
					selection=variation.selection, variation_model=variation.marginalProductModel, mutation=variation.mutation,
					tournamentSize = 2, mutationProb = 'auto',
					randomSeed = 30,
					elitism=elitsm)
			EA.evolve() # Run algorithm
			hypervolume_results.append(EA.hyperVolumeByGeneration)
		hypervolume_results_per_setting.append(hypervolume_results)
	np.save(('data/hypervolume_results_L_' + str(l) + '.npy'), hypervolume_results_per_setting)

#sizes of EA.hyperVolumeByGeneration and EA.numberOfEvaluationsByGeneration are equal
print('hypervolumes:', EA.hyperVolumeByGeneration) #print array of hypervolumes
print('#feval:', EA.numberOfEvaluationsByGeneration) #print array of #feval

 56%|█████▋    | 113/200 [00:22<00:17,  5.10it/s]


KeyboardInterrupt: 

In [ ]:
domain = EA.numberOfEvaluationsByGeneration
for i in range(len(hypervolume_results_per_setting)):
	res = hypervolume_results_per_setting[i]
	interdecile_range_low = np.percentile(res, 10, axis=0)
	interdecile_range_high = np.percentile(res, 90, axis=0)

	average = np.average(res, axis=0)

	plt.plot(domain, average)
	plt.fill_between(domain,
		interdecile_range_low,
		interdecile_range_high,
					alpha=0.3)
plt.legend(L) # [("l = " + x) for x in L]
plt.show()